## Using Dice to segmentation single cell

```{bash}
DATAPATH="/mnt/data/lyx/IMC/batch2/unet/" 
mkdir "$DATAPATH"
```

### download model
```{bash}
mkdir -p "$DATAPATH"/model
wget https://ndownloader.figshare.com/files/28301040 -O "$DATAPATH"model/02-15-20-14_threshold-99.7_withAugnoise-0.5_model_80.pth
```

### preprocess image data
```{bash}
find /mnt/data/lyx/IMC/batch2/fullstacks/ -type f -size -4M
ls -l  | grep "^-" | wc -l

python /mnt/data/lyx/IMC/Dice-XMBD/process/1_preprocess_img.py --process pre --workdir $DATAPATH --panel "/mnt/data/lyx/IMC/batch2/IMC_CRC_panel_Batch2.csv" --imgsuf '_fullstacks.tiff' --imgdir "/mnt/data/lyx/IMC/batch2/fullstacks/" --channel_n "channel_n" --channel_m "channel_m"
```

### train a model (optional)
```{bash}
python ./main_probability.py --action train --epoch 30 --workdir "$DATAPATH"BRCA1 --bs 2
```

### predict pixel probability maps by a trained model
```{bash}
python /mnt/data/lyx/IMC/Dice-XMBD/src/main_probability.py --action predict --weight="${DATAPATH}model/02-15-20-14_threshold-99.7_withAugnoise-0.5_model_80.pth" --preddir "${DATAPATH}" --testname "crop512_combined2_image"
```

### generate cell masks by using Cellprofiler pipeline ('/workspace/process/2_generate_cellmask.cpproj')

### merge figure back (cell mask) (output from CellProfiler)
```{bash}
python /mnt/data/lyx/IMC/Dice-XMBD/process/1_preprocess_img.py --process post --workdir $DATAPATH --propn "cellmask" --mergetype 'cmask' --outdir 'ori_cellmask'
python /mnt/data/lyx/IMC/Dice-XMBD/process/1_preprocess_img.py --process post --workdir $DATAPATH --propn "cellmask" --mergetype 'cmask_tight' --outdir 'ori_cellmask_tight' (another mode, refer to readme)
```
### extract single cell protein
#### (if using IMCell to denose, skip this step)
```{bash}
python3 /mnt/data/lyx/IMC/Dice-XMBD/process/3_imc_extract_pro.py --workdir $DATAPATH --imgdir "/mnt/data/lyx/IMC/fullstacks/" --maskpath "$DATAPATH"predictionProbability/ori_cellmask/ --name 'ori' --imgsuf '_fullstacks.tiff' --masksuf '_pred_Probabilities_cell_mask.tiff' --panel "/mnt/data/lyx/IMC/IMC_CRC_panel.csv" --match 'F'

python3 /mnt/public/temp/xiaoxu/IMC/src/unet/3_imc_extract_pro.py --workdir $DATAPATH --imgdir '/mnt/public/temp/xiaoxu/IMC/chang/analysis_0401/fullstacks/' --maskpath "$DATAPATH"predictionProbability/ori_cellmask_tight/ --name 'ori' --imgsuf '_fullstacks.tiff' --masksuf '_pred_Probabilities_cell_mask_tight.tiff' --panel '/mnt/public/temp/xiaoxu/IMC/chang/IMC_CRC_panel.csv' --match 'F'
```

## IMCell Denoise

#### IMCell main and plot
```{bash}
DATAPATH="/mnt/data/lyx/IMC/batch1/"

python /mnt/data/lyx/IMC/IMCell/main.py --workdir $DATAPATH  --img_path "$DATAPATH"fullstacks --mask_path "$DATAPATH"unet/predictionProbability/ori_cellmask --panel "$DATAPATH"IMC_CRC_panel_Batch1.csv --qc "$DATAPATH"IMC_CRC_QC_Batch1.csv --times 1000 --fdr_value 0.01 --pre 1 --cellinfo 1 --permutation 1 --denoise 1 --norm 1
```